In [1]:
import os
os.chdir('../..')

In [2]:
from smart_evidence.helpers.data import hash_document

In [3]:
import srsly
from glob import glob

ModuleNotFoundError: No module named 'srsly'

In [ ]:
from haystack.document_stores import OpenSearchDocumentStore
from smart_evidence.helpers import opensearch_connection


# DocumentStore: holds all your data
document_store = OpenSearchDocumentStore(
    username="admin",
    password="R9$Cix3vD$BU#z",
    host=opensearch_connection.HOST,
    port=443,
    timeout=600,
    aws4auth=opensearch_connection.AWS_AUTH,
    verify_certs=True,
    index="paragraphs_v4",
    label_index="haystack-paragraphs-labels",
    search_fields=["text", "title"],
    embedding_field="embedding",
    similarity="cosine",
    excluded_meta_data=[
        "paragraphs",
    ],
    content_field="text",
    name_field="title",
)


In [ ]:
def delete_by_ids(ids):
    query = {"query": {"terms": {"document_id": ids}}, '_source': False}
    res = document_store.client.delete_by_query(index=document_store.index, body=query, requests_per_second=10, timeout='10m', _source=False)

In [ ]:
from smart_evidence.helpers.concept_patterns import get_keywords_from_yamls
impact_keywords = set([k['keyword'] for k in get_keywords_from_yamls(['assets/keywords_clean/IMPACT.yaml'])])
company_keywords = set([k['keyword'] for k in get_keywords_from_yamls(['assets/keywords_clean/COMPANY.yaml'])])

In [ ]:
len(impact_keywords), len(company_keywords)

In [ ]:
for documents_path in sorted(glob('corpus/future_documents/wikipedia-*.jsonl')):
    to_delete = 0
    to_stay = 0
    document_ids_to_delete = []
    for document in srsly.read_jsonl(documents_path):
        if (document['title'] in impact_keywords) or (document['title'] in company_keywords):
            to_stay += 1
        else:
            document_ids_to_delete.append(hash_document(document))
    print(documents_path, len(document_ids_to_delete), to_stay)
    while len(document_ids_to_delete):
        delete_by_ids(document_ids_to_delete[:1000])
        document_ids_to_delete = document_ids_to_delete[1000:]

In [ ]:
query = {"query": {"terms": {"document_id": document_ids_to_delete}}, '_source': False}
res = document_store.client.delete_by_query(index=document_store.index, body=query, requests_per_second=1, timeout='10m', _source=False)

In [ ]:
len(document_ids_to_delete)